In [ ]:
import numpy as np
import pandas as pd
import pickle
import sys
import torch

from tqdm.notebook import tqdm
from typing import Dict, List

sys.path.append("../seal_counter/Notebooks/RCNN Notebooks")
from rcnn_utils import (
    decode_prediction, 
    get_bb, 
    get_object_detection_model,
    predict, 
    write_to_latex
)

In [ ]:
# Connect to the GPU if one exists.
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using: ", device)
torch.cuda.empty_cache()

In [ ]:
# Load CNN predictions
training_pred_path = "../seal_detector/Generated Data/training_seals_pytorch.pkl"
val_pred_path = "../seal_detector/Generated Data/validation_seals_pytorch.pkl"
test_pred_path = "../seal_detector/Generated Data/testing_seals_pytorch.pkl"

with open(training_pred_path, "rb") as f:
    train_preds = pickle.load(f)
with open(val_pred_path, "rb") as f:
    val_preds = pickle.load(f)
with open(test_pred_path, "rb") as f:
    test_preds = pickle.load(f)

In [ ]:
# Load models
rcnn_unfrozen = get_object_detection_model(path="../seal_counter/Models/rcnn_extra_data_base_30_10", version=1)
rcnn_frozen_v1 = get_object_detection_model(path="../seal_counter/Models/rcnn_trial1_50", version=1)
rcnn_frozen_v2 = get_object_detection_model(path="../seal_counter/Models/rcnn_trial3_50", version=2)

In [ ]:
# Load path to images
training_image_path = "../Training, Val, and Test Images/Training Images/"
validation_image_path = "../Training, Val, and Test Images/Validation Images/"
testing_image_path = "../Training, Val, and Test Images/Test Images/"

In [ ]:
def compare_models(models:List, model_names:List[str], preds_dict:Dict, nms_thresh:float, score_thresh:float, xml_path:str) -> pd.DataFrame:
    """Compares the count of the provided models for a specific image

    Args:
        models (List): List of pytorch models to compare
        model_names (List[str]): Names of the models
        preds_dict (Dict): Dictionary mapping image file name to RCNN predictions for that image
        nms_thresh (float): NMS thresold
        score_thresh (float): Score threshold
        xml_path (str): path to image XML

    Returns:
        pd.DataFrame: DataFrame containing actual and predicted seal counts for each model
    """
    
    file_names = [file_name for file_name in preds_dict.keys()]
    counts = [[] for _ in models]
    actual_count = []

    # Generate counts for each image in dataset
    for file_name in tqdm(file_names):

        # Initialize count for each model to 0 and get actual count
        image_count = [0 for _ in models]
        actual_count.append(
            get_bb(xml_path, [file_name+".xml"]).shape[0]
            )
        
        # Iterate through  each sub-image per iamge
        for sub_img in preds_dict[file_name]:
            
            # Get prediction for specific sub-image for each model
            for i in range(len(image_count)):
                pred = decode_prediction(predict(models[i], sub_img), score_thresh, nms_thresh)
                image_count[i] += len(pred)
        
        # Update list of counts for image with model counts
        for i in range(len(image_count)):
            counts[i].append(image_count[i])

    # DataFrame containing counts
    df = pd.DataFrame({"File Name":file_names, "Actual Count": actual_count})

    # Printing Metrics
    for i in range(len(model_names)):
        
        # Constants
        model_name = model_names[i]
        df[model_name] = counts[i]
        absolute_difference = abs(df["Actual Count"] - df[model_name])

        # Metric Calculation
        mean_absolute_percent_error = (absolute_difference / df["Actual Count"]).mean()
        mean_absolute_error = absolute_difference.mean()
        error_per_ten_seals = (mean_absolute_error * 10) / df["Actual Count"].mean()
        total_miscounted_seals = absolute_difference.sum()
   
        # Print Statements
        print(f"Metrics for model: {model_name}")

        name_to_metric_map = {
            "Mean Absolute Percent Error": mean_absolute_percent_error,
            "Mean Absolute Error": mean_absolute_error,
            "Error per 10 seals": error_per_ten_seals,
            "Total Miscounted Seals" : total_miscounted_seals
        }
        max_string_length = max(
            [len(metric_name) for metric_name in name_to_metric_map.keys()]
        )
        for metric_name in name_to_metric_map.keys():
            print(f"\t{metric_name:<{max_string_length + 1}}: {name_to_metric_map[metric_name]:>5.4f}")

    return df


In [ ]:
# Initialize model lists
unfrozen_model_list = [rcnn_unfrozen,]
unfrozen_model_names = ["Unfrozen V1"]

frozen_model_list = [ rcnn_frozen_v1, rcnn_frozen_v2]
frozen_model_names = [ "Frozen V1", "Frozen V2"]

In [ ]:
def evaluate_count(
        unfrozen_model_list:List, 
        unfrozen_model_names:List[str], 
        frozen_model_list:List, 
        frozen_model_names:List[str], 
        preds:Dict, 
        image_path:str, 
        nms_thresh:float=.1, 
        unfrozen_score_thresh:float=.75, 
        frozen_thresh:float=.65
        ) -> pd.DataFrame:
    """Creates Dataframe containing true and predicted counts of specified frozen and unfrozen models.
       Unfrozen and frozen models can use different scores.

    Args:
        unfrozen_model_list (List): List of unfrozen pytorch models
        unfrozen_model_names (List[str]): List of unfrozen model names
        frozen_model_list (List): List of frozen pytorch models
        frozen_model_names (List[str]): LList of frozen model names
        preds (Dict): Dictionary mapping image name to RCNN predictions for dataset
        image_path (str): Path to images/xml files in the dataset
        nms_thresh (float, optional): NMS threshold. Defaults to .1.
        unfrozen_score_thresh (float, optional): Score threshold to be used for unfrozen models. Defaults to .75.
        frozen_thresh (float, optional): Score threshold to be used for frozen models. Defaults to .65.

    Returns:
        pd.DataFrame: Dataframe containing actual and predicted counts for all models specified
    """
    
    # Get df for unfrozen models
    unfrozen_results = compare_models(
            unfrozen_model_list, 
            unfrozen_model_names, 
            preds, 
            nms_thresh, 
            unfrozen_score_thresh, 
            image_path,
        )
    
    # Get df for frozen models 
    frozen_results = compare_models(
            frozen_model_list, 
            frozen_model_names, 
            preds, 
            nms_thresh, 
            frozen_thresh,
            image_path,
        )
    
    # Combine dfs
    merge_columns = ["File Name", "Actual Count"]
    return unfrozen_results.merge(frozen_results, right_on=merge_columns, left_on=merge_columns).sort_values(by="File Name")

### Training

In [ ]:
training_results = evaluate_count(unfrozen_model_list, unfrozen_model_names, frozen_model_list, frozen_model_names, train_preds, training_image_path)

In [ ]:
training_results

### Validation

In [ ]:
validation_results = evaluate_count(unfrozen_model_list, unfrozen_model_names, frozen_model_list, frozen_model_names, val_preds, validation_image_path)

In [ ]:
validation_results

### Testing

In [ ]:
test_results = evaluate_count(unfrozen_model_list, unfrozen_model_names, frozen_model_list, frozen_model_names, test_preds, testing_image_path)

In [ ]:
test_results

In [ ]:
training_results.to_csv("training_results.csv", index=False)
validation_results.to_csv("validation_results.csv", index=False)
test_results.to_csv("test_results.csv", index=False)

In [ ]:
# Save to latex (if you want to put the results in a paper)

# write_to_latex(training_results, "training_results", long_table=True)
# write_to_latex(validation_results, "validation_results", long_table=True)
# write_to_latex(test_results, "test_results", long_table=True)